In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3

conn = sqlite3.connect('example.db')


def rsq(query: str) -> pd.DataFrame:
    return pd.read_sql_query(query, conn)


In [5]:
import pyodbc

# create a connection to the database
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=server_name;DATABASE=database_name;UID=username;PWD=password')

# create a cursor object
cursor = conn.cursor()

# execute the SQL code
cursor.execute("""DECLARE @counter INTEGER
SET @counter = 20
SELECT @counter;""")

# fetch the results
result = cursor.fetchone()

# print the result
print(result[0])

# close the cursor and connection
cursor.close()
conn.close()


DatabaseError: Execution failed on sql ' -- Declare the variable (a SQL Command, the var name, the datatype)
DECLARE @counter INTEGER

-- Set the counter to 20
SET @counter = 20

-- Select the counter
SELECT @counter;': near "DECLARE": syntax error

In [ ]:
"""-- Declare the variable (a SQL Command, the var name, the datatype)
DECLARE @counter INT 

-- Set the counter to 20
SET @counter = 20

-- Select and increment the counter by one 
SELECT @counter = @counter + 1

-- Print the variable
SELECT @counter"""

In [ ]:
"""DECLARE @counter INT 
SET @counter = 20

-- Create a loop
WHILE @counter < 30

-- Loop code starting point
BEGIN
	SELECT @counter = @counter + 1
-- Loop finish
END

-- Check the value of the variable
SELECT @counter"""

In [15]:
rsq("""SELECT a.RecordId, a.Age, a.BloodGlucoseRandom, 
  -- Select maximum glucose value (use colname from derived table)    
  b.MaxGlucose
FROM kidney a
-- Join to derived table
INNER JOIN (SELECT Age, MAX(BloodGlucoseRandom) AS MaxGlucose FROM Kidney GROUP BY Age) b
ON a.Age = b.Age; """)

,RecordID,Age,BloodGlucoseRandom,MaxGlucose
0,1,48,121,214
1,1,7,0,0
2,2,62,423,423
3,3,48,117,214
4,4,51,106,129
...,...,...,...,...
395,374,55,140,297
396,375,42,75,132
397,376,12,100,100
398,377,17,114,114


In [17]:
rsq("""SELECT *
FROM Kidney a
-- JOIN and create the derived table
JOIN (SELECT Age, MAX(BloodPressure) AS MaxBloodPressure FROM Kidney GROUP BY Age) b
-- JOIN on BloodPressure equal to MaxBloodPressure
ON a.BloodPressure = b.MaxBloodPressure
-- Join on Age
AND a.Age = b.Age""")

,Age,BloodPressure,SpecificGravity,Albumin,Sugar,RedbloodCells,PusCell,PuscellClumps,bacteria,BloodGlucoseRandom,...,Hypertension,DiabetesMellitus,CoronaryArteryDisease,Appetite,PedalEdema,Anemia,Class,RecordID,Age,MaxBloodPressure
0,7,50,1.020,4,0,None,normal,notpresent,notpresent,0,...,None,None,None,good,no,no,ckd,1,7,50
1,52,100,1.015,3,0,normal,abnormal,present,notpresent,138,...,yes,yes,None,good,no,yes,ckd,9,52,100
2,40,80,1.015,3,0,None,normal,notpresent,notpresent,76,...,yes,None,None,good,no,yes,ckd,15,40,80
3,60,100,1.025,0,3,None,normal,notpresent,notpresent,263,...,yes,yes,yes,good,no,no,ckd,18,60,100
4,42,100,1.015,4,0,normal,abnormal,notpresent,present,0,...,yes,None,None,poor,no,no,ckd,24,42,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,80,80,1.025,0,0,normal,normal,notpresent,notpresent,119,...,None,None,None,good,no,no,notckd,362,80,80
102,15,80,1.025,0,0,normal,normal,notpresent,notpresent,93,...,None,None,None,good,no,no,notckd,366,15,80
103,41,80,1.025,0,0,normal,normal,notpresent,notpresent,112,...,None,None,None,good,no,no,notckd,368,41,80
104,36,80,1.025,0,0,normal,normal,notpresent,notpresent,85,...,None,None,None,good,no,no,notckd,370,36,80


In [18]:
rsq("""-- Specify the keyowrds to create the CTE
WITH BloodGlucoseRandom AS (SELECT MAX(BloodGlucoseRandom) AS MaxGlucose FROM Kidney)

SELECT a.Age, b.MaxGlucose
FROM Kidney a
JOIN BloodGlucoseRandom b ON a.BloodGlucoseRandom = b.MaxGlucose;

""")

,Age,MaxGlucose
0,50,490
1,60,490


In [19]:
rsq("""-- Create the CTE
WITH BloodPressure (MaxBloodPressure) 
AS (SELECT MAX(BloodPressure) AS MaxBloodPressure FROM Kidney)

SELECT *
FROM Kidney a
-- Join the CTE  
JOIN BloodPressure b
ON a.BloodPressure = b.MaxBloodPressure""")

,Age,BloodPressure,SpecificGravity,Albumin,Sugar,RedbloodCells,PusCell,PuscellClumps,bacteria,BloodGlucoseRandom,...,RedBloodCellCount,Hypertension,DiabetesMellitus,CoronaryArteryDisease,Appetite,PedalEdema,Anemia,Class,RecordID,MaxBloodPressure
0,56,180,None,0,4,None,abnormal,notpresent,notpresent,298,...,4.2,yes,yes,None,poor,yes,no,ckd,96,180
